CARGA INCREMENTAL (este paso requiere que previeamente se realice la creación del modelo relacional en workbench)

In [ ]:
#establecimiento de conexión con la base de datos pi_1
import pandas as pd
import numpy as np
import pymysql
import sqlalchemy as db
database_username='root' # Nombre de cliente en MySQL Workbench
database_password='4488' # Contraseña de MySQL Workbench
database_ip='localhost' # localizacion del servidor
database_name='pi_1' # Nombre de Base de datos a la que nos conectaremos
engine=db.create_engine(f'mysql+pymysql://{database_username}:{database_password}@{database_ip}/{database_name}')
conexion=engine.connect() 

In [ ]:
#Transformacion de dataset basado en las transformaciones de los datos iniciales

precios_nuevos= pd.read_csv('Datasets/precios_semana_20200518.txt', sep="|") #lectura

data_types_dict = {'producto_id':str , 'sucursal_id':str}
precios_nuevos = precios_nuevos.astype(data_types_dict) # define dtype 

def quitar_guion(codigo):  # esta es una funcion para quitar los '-' de las columnas producto_id
    y= codigo.replace("-","")
    return y 

precios_nuevos['producto_id']= precios_nuevos['producto_id'].map(quitar_guion) # elimina guiones de ids de producto
precios_nuevos.head(3)

precios_nuevos['producto_id'] = precios_nuevos['producto_id'].apply(lambda x : np.NaN if x=='nan' else x) # asigna NAN a campos precio vacios
precios_nuevos['precio'] = precios_nuevos['precio'].apply(lambda x : None if x=='' else x) # asigna NAN a campos precio vacios

data_types_dict = {'producto_id':float , 'precio':float} # cambia tipo a float para poder trabajar con NAN
precios_nuevos = precios_nuevos.astype(data_types_dict) 

precios_nuevos = precios_nuevos.reindex(columns=['precio','producto_id','sucursal_id']) # ordenar columnas

precios_nuevos['sucursal_id'] = precios_nuevos['sucursal_id'].apply(lambda x : np.NaN if x=='nan' else x) # convierte 'nan' de columna sucursal_id en np.NAN

precios_nuevos.dropna(subset=['precio','sucursal_id'], inplace=True) # elimina los que no tienen precio o sucursal_id

precios_nuevos.drop_duplicates(inplace=True) # se elimina duplicados

precios_nuevos.columns=['precio', 'id_producto', 'id_sucursal'] # se cambian los nombre de las columnas

sucursales_db=pd.read_sql_table(table_name='sucursales', con=conexion) # trae la tabla sucursales
merge_pnus=pd.merge(precios_nuevos , sucursales_db , how='left' , on='id_sucursal') #
merge_pnus.loc[:,'id_sucursal'][merge_pnus.id_comercio.isnull()] = None
precios_nuevos = merge_pnus.loc[:,['precio','id_producto','id_sucursal']]

In [ ]:
types_precio_dict={'precio':db.types.FLOAT, 'id_producto':db.types.BIGINT, 'id_sucursal':db.types.VARCHAR(length=10)}
precios_nuevos.to_sql('precios',conexion, index=False , dtype=types_precio_dict, if_exists='append') #carga incremental a la tabla precios de la db pi_1

In [ ]:
conexion.close()